In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForQuestionAnswering
import gc

In [2]:
# tokeniser = BertTokenizer.from_pretrained(model_dir)  # Add specific options if needed
model = BertForQuestionAnswering.from_pretrained('bert-base-chinese')
tokeniser = BertTokenizer.from_pretrained('bert-base-chinese')

model.eval()

if torch.cuda.is_available():

    dirve ='cuda'
else:
    dirve ='cpu'
print('use ',dirve)
model.to(dirve)
print('加载模型完成')

100%|██████████| 109540/109540 [00:00<00:00, 4137421.07B/s]


use  cpu
加载模型完成


In [13]:
#释放内存
# del model
# gc.collect()

44

In [1]:
# model

In [3]:
q='明天天气怎么样'

context="""

明天天气不错 我们不如一起出去吧
"""
 


question='[CLS] '+ q + ' [SEP]'

context= context+ ' [SEP]'

len_line_1 = len(tokeniser.tokenize(question))
len_line_2 = len(tokeniser.tokenize(context))


input_mask =question + context

# text = question + context

#         print('text',text)
tokenized_text = tokeniser.tokenize(input_mask)
print('tokenized_text',tokenized_text)

indexed_tokens = tokeniser.convert_tokens_to_ids(tokenized_text)
# input_mask_tokens = tokeniser.convert_tokens_to_ids(input_mask)
segments_ids = ([0] * len_line_1) + ([1] * len_line_2)
input_mask_tokens = [1]*len(indexed_tokens)

print(torch.long)
print(indexed_tokens)

input_mask_tensors= torch.tensor([input_mask_tokens], dtype=torch.long).to(dirve)
tokens_tensor = torch.tensor([indexed_tokens], dtype=torch.long).to(dirve)
segments_tensors = torch.tensor([segments_ids], dtype=torch.long).to(dirve)

print('tokens_tensor',tokens_tensor)
print('segments_tensors',segments_tensors)
print('input_mask_tensors',input_mask_tensors)

#         print('tokens_tensor',tokens_tensor)
#         print('segments_tensors',segments_tensors)
#这里使用模型进行预测
batch_start_logits, batch_end_logits = model(tokens_tensor, segments_tensors,input_mask_tensors)
print('batch_start_logits',batch_start_logits)
print('batch_end_logits',batch_end_logits)

tokenized_text ['[CLS]', '明', '天', '天', '气', '怎', '么', '样', '[SEP]', '明', '天', '天', '气', '不', '错', '我', '们', '不', '如', '一', '起', '出', '去', '吧', '[SEP]']
torch.int64
[101, 3209, 1921, 1921, 3698, 2582, 720, 3416, 102, 3209, 1921, 1921, 3698, 679, 7231, 2769, 812, 679, 1963, 671, 6629, 1139, 1343, 1416, 102]
tokens_tensor tensor([[ 101, 3209, 1921, 1921, 3698, 2582,  720, 3416,  102, 3209, 1921, 1921,
         3698,  679, 7231, 2769,  812,  679, 1963,  671, 6629, 1139, 1343, 1416,
          102]])
segments_tensors tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])
input_mask_tensors tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])
batch_start_logits tensor([[-0.2314, -0.3325, -0.4691, -0.1570, -0.1176, -0.3229, -0.2685,  0.0176,
          0.1297, -0.5480, -0.4350, -0.1237, -0.0902,  0.0382,  0.1981, -0.1992,
         -0.2722, -0.3234, -0.3967, -0.4319, -0.5689, -0.1490, -0.4177, -0.1311,
          0

In [4]:
def get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)
  print('index_and_score',index_and_score)

  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes

In [12]:
n_best_size = 10

In [6]:
start_logits = batch_start_logits[0].detach().cpu().tolist()
start_logits

[-0.23139816522598267,
 -0.3325277268886566,
 -0.4690726101398468,
 -0.1570422649383545,
 -0.11756917834281921,
 -0.3228791654109955,
 -0.2684593200683594,
 0.01763373613357544,
 0.129669189453125,
 -0.5479621291160583,
 -0.4350026547908783,
 -0.12372050434350967,
 -0.09021744132041931,
 0.03824535012245178,
 0.1981472223997116,
 -0.1992499679327011,
 -0.27217161655426025,
 -0.32344263792037964,
 -0.39665329456329346,
 -0.43190065026283264,
 -0.5689064264297485,
 -0.14896880090236664,
 -0.4176713228225708,
 -0.13106927275657654,
 0.12966956198215485]

In [7]:
end_logits = batch_end_logits[0].detach().cpu().tolist()
end_logits

[0.597241997718811,
 0.4694574177265167,
 0.5503278970718384,
 0.6202974319458008,
 0.31571725010871887,
 0.4575151801109314,
 0.5127164125442505,
 0.9626365900039673,
 0.22147777676582336,
 0.7128044366836548,
 0.519715428352356,
 0.4107130169868469,
 0.22277796268463135,
 0.40070706605911255,
 0.08105812966823578,
 0.7070112228393555,
 0.3245834410190582,
 0.6371434330940247,
 0.5859260559082031,
 0.7214884757995605,
 0.6194947957992554,
 1.2890911102294922,
 0.7240879535675049,
 0.5007027387619019,
 0.22147813439369202]

In [8]:
start_indexes = get_best_indexes( start_logits, n_best_size)
end_indexes = get_best_indexes(end_logits, n_best_size)


print('start_indexes',start_indexes)
print('end_indexes',end_indexes)

index_and_score [(14, 0.1981472223997116), (24, 0.12966956198215485), (8, 0.129669189453125), (13, 0.03824535012245178), (7, 0.01763373613357544), (12, -0.09021744132041931), (4, -0.11756917834281921), (11, -0.12372050434350967), (23, -0.13106927275657654), (21, -0.14896880090236664), (3, -0.1570422649383545), (15, -0.1992499679327011), (0, -0.23139816522598267), (6, -0.2684593200683594), (16, -0.27217161655426025), (5, -0.3228791654109955), (17, -0.32344263792037964), (1, -0.3325277268886566), (18, -0.39665329456329346), (22, -0.4176713228225708), (19, -0.43190065026283264), (10, -0.4350026547908783), (2, -0.4690726101398468), (9, -0.5479621291160583), (20, -0.5689064264297485)]
index_and_score [(21, 1.2890911102294922), (7, 0.9626365900039673), (22, 0.7240879535675049), (19, 0.7214884757995605), (9, 0.7128044366836548), (15, 0.7070112228393555), (17, 0.6371434330940247), (3, 0.6202974319458008), (20, 0.6194947957992554), (0, 0.597241997718811), (18, 0.5859260559082031), (2, 0.5503278

In [16]:
print('tokenized_text',tokenized_text)

text = tokenized_text[1:3]
print('预测结果',text)
max_answer_length=20
for start_index in start_indexes:
    for end_index in end_indexes:
        # We could hypothetically create invalid predictions, e.g., predict
        # that the start of the span is in the question. We throw out all
        # invalid predictions.
                    
#         print('*'*20)
#         print('start_index',start_index)
#         print('end_index',end_index)
        start= int(start_index)
        end =int(end_index)
        if start > end:
#             print('不合适')
            continue
        if start < len_line_1 :
#             print('不合适')
            continue

        if end_index >= len(tokenized_text):
            continue
#         if start_index not in feature.token_to_orig_map:
#             continue
#         if end_index not in feature.token_to_orig_map:
#             continue
#         if not feature.token_is_max_context.get(start_index, False):
#             continue
#         if end_index < start_index:
#             continue
        length = end_index - start_index + 1
        if length > max_answer_length:
            continue
        else:
            if start_logits[start]<0:
                continue
            if start_logits[start]<0.1:
                continue
            if end_logits[end]<0:
                continue
            if end_logits[end]<0.1:
                continue
            text = tokenized_text[start:end]
            print('预测结果',text)
            key=lambda x: (x.start_logits[start] + x.start_logits[start])
            print("key",key)
            
            print("start_logits",start_logits[start])
            print("end_logits",end_logits[end])
            

        
#         if start_index >= len(indexed_tokens):
#             continue
#         if end_index >= len(indexed_tokens):
#             continue
#         if start_index not in feature.token_to_orig_map:
#             continue
#         if end_index not in feature.token_to_orig_map:
#             continue
#         if not feature.token_is_max_context.get(start_index, False):
#             continue
#         if end_index < start_index:
#             continue
#         length = end_index - start_index + 1
#         if length > max_answer_length:
#             continue
#         prelim_predictions.append(
#             _PrelimPrediction(
#                 feature_index=feature_index,
#                 start_index=start_index,
#                 end_index=end_index,
#                 start_logit=result.start_logits[start_index],
#                 end_logit=result.end_logits[end_index]))

tokenized_text ['[CLS]', '明', '天', '天', '气', '怎', '么', '样', '[SEP]', '明', '天', '天', '气', '不', '错', '我', '们', '不', '如', '一', '起', '出', '去', '吧', '[SEP]']
预测结果 ['明', '天']
预测结果 ['错', '我', '们', '不', '如', '一', '起']
key <function <lambda> at 0x7f55331170d0>
start_logits 0.1981472223997116
end_logits 1.2890911102294922
预测结果 ['错', '我', '们', '不', '如', '一', '起', '出']
key <function <lambda> at 0x7f5533117ea0>
start_logits 0.1981472223997116
end_logits 0.7240879535675049
预测结果 ['错', '我', '们', '不', '如']
key <function <lambda> at 0x7f5533117d08>
start_logits 0.1981472223997116
end_logits 0.7214884757995605
预测结果 ['错']
key <function <lambda> at 0x7f5533117950>
start_logits 0.1981472223997116
end_logits 0.7070112228393555
预测结果 ['错', '我', '们']
key <function <lambda> at 0x7f5533117048>
start_logits 0.1981472223997116
end_logits 0.6371434330940247
预测结果 ['错', '我', '们', '不', '如', '一']
key <function <lambda> at 0x7f5533117a60>
start_logits 0.1981472223997116
end_logits 0.6194947957992554
预测结果 ['错', '我', '们', 

In [7]:
batch_start_logits[0, 0]

tensor(0.2047, device='cuda:0', grad_fn=<SelectBackward>)